In [44]:
import pandas as pd
import numpy as np
import math
import time
import os
import requests
import json
import urllib.request
import re
import pickle
import networkx as nx
import string

In [54]:
app_id = '24ac8855'
app_key = 'baa032bb2944756dcf361a18ae7e3ab9'

actual_dictionary = {}

alphabet = set(list(string.ascii_lowercase) + list(string.ascii_uppercase))

def find_defs(diction):
    """
    This function executes when we reach a list containing dictionary data strucutres that may contain several different
    definitions for a given word. It runs through this list, for each list finding keys in the dictionary data
    structure containing the string "definitions" or "short_definitions" and appending the corresponding value 
    (a string) of this key to a master string, which we call stringy. At the end of this function, we use the 
    built-in function set() on the list given to us by the .split() method. The reasons for a set are:
    1) checking membership in sets is efficient in python
    2) sets do not contain duplicate elements, so we are using less space.
    """
    stringy = ''
    for i in diction:
        for k, v in i.items():
            if k == 'definitions' or k == 'short_definitions' or k == 'crossReferenceMarkers':
                stringy += v[0] + ' '
    stringy = stringy.split(' ')
    
    for word in stringy:  
        n = stringy.index(word)
        try:
            # Make sure that first, last element in string is an alphanumeric character
            if word[0] not in alphabet:
                stringy[n] = word[1:]
            if stringy[n][-1] not in alphabet:
                stringy[n] = stringy[n][:-1]
            # sometimes dictionary entries have phrases like "(hello it is me.)
            # this has two non-alphanumeric characters at the end. This check below catches this.
            if stringy[n][-1] not in alphabet:
                stringy[n] = stringy[n][:-1]
        except IndexError:
            pass

    return set(stringy)

def find_definition(word):
    """
    This function takes in a word as input and creates an entry in our dataset consisting of a word-definition pair.
    """
#     wordnet_lemmatizer.lemmatize(word)
    
    # there may be times while looking up words that the dictionary does not contain a word for whatever reason.
    # this chunk of code catches this error.
    try:
#         lemmatizedWord = wordnet_lemmatizer.lemmatize(word).lower()
        url = 'https://od-api.oxforddictionaries.com:443/api/v1/entries/en/' + word.lower()
        r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
        dicty = r.json()
        # # Need to consider words like "won" that have atypical structure...........       
        # this process is ugly, but it works. 
        # The structure of the data returned from the API call is a bit awkward.
        lexCategory = dicty['results'][0]['lexicalEntries'][0]['lexicalCategory']
        if lexCategory == "Noun":
            lexCategory = "n"
        if lexCategory == "Verb":
            lexCategory = "v"
        if lexCategory == "Adjective":
            lexCategory = "a"
        definitions = dicty['results'][0]['lexicalEntries'][0]['entries'][0]['senses']
        if find_defs(definitions) != {''}:
            actual_dictionary[word] = find_defs(definitions)
        else:
            pass
    except json.JSONDecodeError:
        pass
    
def find_contents(diction):
    try:
        for k, v in diction.items():
            print(k, '\n', v, '\n\n')
    except AttributeError:
        for i in diction:
            print(i, '\n\n')

In [56]:
# Defining functions using "Pickle" library that allow us to save python objects as .pkl files that, when loaded again,
# act just like the objects we saved them as without needing to transform them.

# This allows us to save a dictionary instead of needing to reconsruct a dictionary for each info-theoretic calculation.

def save_obj(obj, name):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [58]:
# get word list to be used for dictionary lookup
word_list = [word.strip('\n') for word in open("data/words_alpha.txt", 'r').readlines()]

In [59]:
for word in word_list:
    try:
        find_definition(word)
    except KeyError:
        pass
    # add delay so we don't reach API call/min ceiling.
    time.sleep(.2)
    save_obj(actual_dictionary, 'dictionary')

save_obj(actual_dictionary, 'dictionary')

In [62]:
dicty = load_obj('dictionary')

In [90]:
# find_contents(dicty)

Dictionaries can be represented as directed graphs D = (V, A) (digraphs). The vertices
(i.e. the members of v) are words and the arcs connect defining words to defined words,
that is, there is an arc from word u to word v if u is a word in the definition of v. More-
over, in a complete dictionary, every word is defined by at least one word, so we assume
that there is no word without an incoming arc. A path is a sequence (v 1 , v 2, ..., v k ) of ver-
tices such that (v i , v i+1 ) is an arc for i = 1, 2, ..., k-1. A circuit is a path starting and end-
ing at the same vertex. A graph is called acyclic if it does not contain any circuits.

Let U x V be any subset of words and let u be some given word. We are interested
in computing all words that can be learned through definitions composed only of words
in U. This can be stated recursively as follows: We say that u is definable from U if
all predecessors of u either belong to U or are definable from U. The set of words that
can be defined from U is denoted by Def(U). In particular, if Def (U)∪U = V, then U
is called a grounding set of D. Intuitively, a set U is a grounding set if, provided we
already know the meaning of each word in U, we can learn the meaning of all the
remaining words just by looking up the definitions of the unknown words (in the right
order).

As a first step, we observed that in all dictionaries analyzed so far there exist many
words that are never used in any definition. These words can be removed without chang-
ing the MinSets. This reduction can be done iteratively until no further word can be
removed without leaving any word undefinable from the rest. The resulting subgraph is
what we called the dictionary’s (grounding) Kernel

Next, since we are dealing with directed graphs, we can subdivide the words according
to their strongly connected components (SCCs). Two words u and v are strongly connected
if there exists a path from u to v as well as a path from v to u. 

In [63]:
Dictionary = nx.DiGraph()
Dictionary.add_nodes_from(dicty.keys())

In [64]:
len(Dictionary)

85183

In [65]:
def createEdgesFromDefinitions(graph, dictionary):
    # iterate through the dictionary's words and corresponding definitions
    for word_1, definition_1 in dictionary.items():
        # iterate through the remaining words
        for word_2 in dictionary.keys():
            # check if the remaining words define the word we're currently iterating through
            if word_2 in definition_1 and word_1 != word_2:
                # if so, create a directed edge from the word that defines the original word to the original word
                graph.add_edge(word_2, word_1)

createEdgesFromDefinitions(Dictionary, dicty)

In [66]:
len(Dictionary.edges)

869180

In [67]:
words_to_remove = []

# this process will reduce the dictionary graph to the kernel
for word in Dictionary:
    # contained is a flag; we set it to False at first
    contained = False
    # iterate through each edge
    for key, value in dicty.items():
        # if the current word in the iteration is in the the definition of another word,
        # flag contained = True and break out of the innermost loop
        if word == edge[0]:
            contained = True
            break
    # otherwise, add the word to the list of words to remove
    if contained == False:
        words_to_remove.append(word)

Kernel = Dictionary.copy()

# remove words that don't define other words from graph
for word in words_to_remove:
    Kernel.remove_node(word)

In [70]:
save_obj(Dictionary, 'dictionary_graph')
Dictionary_graph = load_obj('dictionary_graph')

save_obj(Kernel, 'kernel_graph')
Kernel_graph = load_obj('kernel_graph')

print(len(Dictionary_graph), len(Dictionary_graph.edges))
print(len(Kernel_graph), len(Kernel_graph.edges))

85183 869180
27935 353180


In [77]:
# find core

Core = nx.DiGraph()

for (word_1, word_2) in Kernel_graph.edges:
        if word_1 in dicty[word_2] and word_2 in dicty[word_1]:
            Core.add_node(word_1)
            Core.add_node(word_2)
            Core.add_edge(word_1, word_2)
            Core.add_edge(word_2, word_1)

In [80]:
save_obj(Core, 'core_graph')
Core_graph = load_obj('core_graph')

print(len(Core), len(Core.edges))

3766 5130


In [ ]:
# Create a json object to represent the graph
nodes = [{'name': i}
            for i in Core.nodes()]
links = [{'source': u[0], 'target': u[1]}
         for u in Core.edges()]
with open('graph.json', 'w') as f:
    json.dump({'nodes': nodes, 'links': links},
              f, indent=4,)

[('a', 'class'),
 ('a', 'conversation'),
 ('a', 'first'),
 ('a', 'for'),
 ('a', 'in'),
 ('a', 'indicate'),
 ('a', 'membership'),
 ('a', 'of'),
 ('a', 'or'),
 ('a', 'people'),
 ('a', 'per'),
 ('a', 'person'),
 ('a', 'someone'),
 ('a', 'something'),
 ('a', 'text'),
 ('a', 'the'),
 ('a', 'thing'),
 ('a', 'time'),
 ('a', 'to'),
 ('a', 'with'),
 ('class', 'a'),
 ('class', 'category'),
 ('class', 'type'),
 ('conversation', 'a'),
 ('conversation', 'informal'),
 ('first', 'a'),
 ('first', 'person'),
 ('for', 'a'),
 ('for', 'benefit'),
 ('for', 'exchange'),
 ('for', 'favour'),
 ('for', 'on'),
 ('for', 'period'),
 ('for', 'place'),
 ('for', 'purpose'),
 ('for', 'reason'),
 ('for', 'respect'),
 ('for', 'series'),
 ('in', 'a'),
 ('in', 'activity'),
 ('in', 'before'),
 ('in', 'by'),
 ('in', 'case'),
 ('in', 'condition'),
 ('in', 'else'),
 ('in', 'inclusion'),
 ('in', 'involvement'),
 ('in', 'language'),
 ('in', 'length'),
 ('in', 'medium'),
 ('in', 'occupation'),
 ('in', 'of'),
 ('in', 'or'),
 ('in

In [82]:
Core_defining_power = {}
Kernel_defining_power = {}

In [84]:
for word in Kernel:
    Kernel_defining_power[word] = 0
    for value in dicty.values():
        if word in value:
            Kernel_defining_power[word] += 1
    Kernel_defining_power[word] /= len(Dictionary)    

find_contents(Kernel_defining_power)

a 
 0.5519293755796344 


aa 
 1.1739431576723054e-05 


aardvark 
 2.347886315344611e-05 


ab 
 1.1739431576723054e-05 


aback 
 1.1739431576723054e-05 


abactinal 
 1.1739431576723054e-05 


abacus 
 4.695772630689222e-05 


abalone 
 2.347886315344611e-05 


abandon 
 0.00016435204207412277 


abandoned 
 0.00019957033680429194 


abandonment 
 7.043658946033833e-05 


abasement 
 1.1739431576723054e-05 


abash 
 1.1739431576723054e-05 


abasia 
 1.1739431576723054e-05 


abatement 
 4.695772630689222e-05 


abatis 
 1.1739431576723054e-05 


abattoir 
 1.1739431576723054e-05 


abaxial 
 1.1739431576723054e-05 


abbatial 
 1.1739431576723054e-05 


abbey 
 5.869715788361528e-05 


abbess 
 2.347886315344611e-05 


abbot 
 7.043658946033833e-05 


abbreviated 
 5.869715788361528e-05 


abbreviation 
 8.217602103706138e-05 


abdicate 
 1.1739431576723054e-05 


abdication 
 1.1739431576723054e-05 


abdomen 
 0.0007748024840637216 


abdominal 
 0.0003991406736085839 


abduce

 0.00041088010518530694 


advisability 
 1.1739431576723054e-05 


advisable 
 9.391545261378444e-05 


advise 
 8.217602103706138e-05 


advised 
 2.347886315344611e-05 


adviser 
 9.391545261378444e-05 


advisory 
 4.695772630689222e-05 


advocaat 
 1.1739431576723054e-05 


advocacy 
 0.0003874012420318608 


advocate 
 0.0013382951997464284 


advowee 
 1.1739431576723054e-05 


advowson 
 2.347886315344611e-05 


adze 
 1.1739431576723054e-05 


aecial 
 1.1739431576723054e-05 


aecidium 
 2.347886315344611e-05 


aeciospore 
 1.1739431576723054e-05 


aecium 
 5.869715788361528e-05 


aedicula 
 1.1739431576723054e-05 


aegirine 
 2.347886315344611e-05 


aegirite 
 1.1739431576723054e-05 


aegithognathous 
 1.1739431576723054e-05 


aeon 
 0.00011739431576723055 


aerate 
 1.1739431576723054e-05 


aerated 
 5.869715788361528e-05 


aeration 
 1.1739431576723054e-05 


aerial 
 0.00022304919995773804 


aeroballistics 
 1.1739431576723054e-05 


aerobic 
 0.0001291337473

 3.5218294730169165e-05 


ameliorate 
 1.1739431576723054e-05 


amenable 
 5.869715788361528e-05 


amend 
 2.347886315344611e-05 


amendment 
 1.1739431576723054e-05 


amends 
 0.0001291337473439536 


amenity 
 1.1739431576723054e-05 


amercement 
 3.5218294730169165e-05 


americana 
 8.217602103706138e-05 


americium 
 1.1739431576723054e-05 


amethyst 
 2.347886315344611e-05 


amicable 
 1.1739431576723054e-05 


amice 
 1.1739431576723054e-05 


amid 
 1.1739431576723054e-05 


amide 
 0.0001056548841905075 


amidships 
 1.1739431576723054e-05 


amygdalin 
 1.1739431576723054e-05 


amygdaloidal 
 1.1739431576723054e-05 


amyl 
 3.5218294730169165e-05 


amylase 
 2.347886315344611e-05 


amyloid 
 1.1739431576723054e-05 


amine 
 7.043658946033833e-05 


amino 
 0.000645668736719768 


amiss 
 4.695772630689222e-05 


amity 
 1.1739431576723054e-05 


amitriptyline 
 1.1739431576723054e-05 


ammonia 
 0.0002582674946879072 


ammoniacal 
 2.347886315344611e-05 


am

 2.347886315344611e-05 


arbalest 
 1.1739431576723054e-05 


arbiter 
 2.347886315344611e-05 


arbitrage 
 1.1739431576723054e-05 


arbitrageur 
 1.1739431576723054e-05 


arbitrary 
 8.217602103706138e-05 


arbitrate 
 2.347886315344611e-05 


arbitration 
 1.1739431576723054e-05 


arbitrator 
 5.869715788361528e-05 


arbor 
 2.347886315344611e-05 


arboreal 
 0.00011739431576723055 


arbour 
 2.347886315344611e-05 


arbutus 
 2.347886315344611e-05 


arc 
 0.0002817463578413533 


arcade 
 7.043658946033833e-05 


arch 
 0.0004930561262223683 


archaeology 
 3.5218294730169165e-05 


archaeological 
 9.391545261378444e-05 


archaeologist 
 1.1739431576723054e-05 


archaic 
 0.002747026988953195 


archaism 
 1.1739431576723054e-05 


archangel 
 3.5218294730169165e-05 


archbishop 
 0.00014087317892067666 


archdeacon 
 2.347886315344611e-05 


archduke 
 1.1739431576723054e-05 


arched 
 0.0002582674946879072 


archegonium 
 2.347886315344611e-05 


archenteron 
 3.

 1.1739431576723054e-05 


bakery 
 1.1739431576723054e-05 


balance 
 0.0006339293051430449 


balanced 
 0.00014087317892067666 


balcony 
 7.043658946033833e-05 


bald 
 8.217602103706138e-05 


balding 
 1.1739431576723054e-05 


baldness 
 3.5218294730169165e-05 


bale 
 2.347886315344611e-05 


baleen 
 5.869715788361528e-05 


ball 
 0.0025357172205721797 


ballad 
 4.695772630689222e-05 


ballade 
 1.1739431576723054e-05 


ballast 
 2.347886315344611e-05 


ballet 
 0.00018783090522756887 


ballistic 
 4.695772630689222e-05 


ballistics 
 3.5218294730169165e-05 


ballistocardiography 
 1.1739431576723054e-05 


balloon 
 0.00018783090522756887 


ballooning 
 2.347886315344611e-05 


ballot 
 5.869715788361528e-05 


ballottement 
 1.1739431576723054e-05 


ballroom 
 8.217602103706138e-05 


balls 
 0.0002582674946879072 


balm 
 2.347886315344611e-05 


balsam 
 8.217602103706138e-05 


balustrade 
 1.1739431576723054e-05 


bamboo 
 0.00011739431576723055 


ban 


...

 0.00017609147365084583 


tithe 
 3.5218294730169165e-05 


tithing 
 1.1739431576723054e-05 


titillating 
 1.1739431576723054e-05 


titivate 
 2.347886315344611e-05 


title 
 0.002347886315344611 


titled 
 2.347886315344611e-05 


titmouse 
 2.347886315344611e-05 


titularly 
 1.1739431576723054e-05 


to 
 0.26052146555063804 


toad 
 0.0001056548841905075 


toadfish 
 2.347886315344611e-05 


toadflax 
 1.1739431576723054e-05 


toady 
 1.1739431576723054e-05 


toadstool 
 9.391545261378444e-05 


toast 
 0.00011739431576723055 


toasted 
 8.217602103706138e-05 


tobacco 
 0.0005869715788361528 


toboggan 
 5.869715788361528e-05 


tod 
 1.1739431576723054e-05 


today 
 8.217602103706138e-05 


toddy 
 1.1739431576723054e-05 


toddle 
 1.1739431576723054e-05 


toe 
 0.00022304919995773804 


toea 
 1.1739431576723054e-05 


toenail 
 7.043658946033833e-05 


toffee 
 3.5218294730169165e-05 


toga 
 1.1739431576723054e-05 


together 
 0.0060927649883192655 


toget

 1.1739431576723054e-05 


ultrasonic 
 5.869715788361528e-05 


ultrasonics 
 1.1739431576723054e-05 


ultrasonography 
 2.347886315344611e-05 


ultrasound 
 9.391545261378444e-05 


ultraviolet 
 0.00019957033680429194 


umbel 
 2.347886315344611e-05 


umbilical 
 7.043658946033833e-05 


umbilicus 
 3.5218294730169165e-05 


umbo 
 2.347886315344611e-05 


umbra 
 1.1739431576723054e-05 


umbrella 
 0.0001056548841905075 


umlaut 
 1.1739431576723054e-05 


umpire 
 8.217602103706138e-05 


un 
 1.1739431576723054e-05 


unabated 
 1.1739431576723054e-05 


unable 
 0.0019017879154291349 


unabridged 
 1.1739431576723054e-05 


unaccented 
 4.695772630689222e-05 


unacceptable 
 0.00027000692626463025 


unacceptably 
 1.1739431576723054e-05 


unaccommodating 
 1.1739431576723054e-05 


unaccompanied 
 0.00011739431576723055 


unaccountable 
 3.5218294730169165e-05 


unaccustomed 
 2.347886315344611e-05 


unacknowledged 
 1.1739431576723054e-05 


unacquainted 
 1.173943

 3.5218294730169165e-05 


unquestioning 
 1.1739431576723054e-05 


unquiet 
 3.5218294730169165e-05 


unravel 
 3.5218294730169165e-05 


unreactive 
 7.043658946033833e-05 


unread 
 1.1739431576723054e-05 


unreal 
 0.0001056548841905075 


unrealistic 
 0.00014087317892067666 


unrealistically 
 3.5218294730169165e-05 


unrealizable 
 1.1739431576723054e-05 


unrealized 
 1.1739431576723054e-05 


unreasonable 
 0.00018783090522756887 


unreasonably 
 0.00011739431576723055 


unreasoning 
 2.347886315344611e-05 


unrebukable 
 1.1739431576723054e-05 


unrebuked 
 2.347886315344611e-05 


unreceptive 
 2.347886315344611e-05 


unreceptiveness 
 1.1739431576723054e-05 


unreclaimable 
 1.1739431576723054e-05 


unrecorded 
 1.1739431576723054e-05 


unrefined 
 9.391545261378444e-05 


unreflective 
 2.347886315344611e-05 


unreformed 
 1.1739431576723054e-05 


unrefreshing 
 1.1739431576723054e-05 


unregenerate 
 2.347886315344611e-05 


unregenerated 
 1.17394315767

 4.695772630689222e-05 


vocational 
 3.5218294730169165e-05 


vocative 
 2.347886315344611e-05 


vociferous 
 2.347886315344611e-05 


vodka 
 5.869715788361528e-05 


voyage 
 5.869715788361528e-05 


voice 
 0.0015496049681274432 


voiced 
 4.695772630689222e-05 


voiceless 
 3.5218294730169165e-05 


void 
 9.391545261378444e-05 


voided 
 2.347886315344611e-05 


voyeurism 
 1.1739431576723054e-05 


voyeuristic 
 1.1739431576723054e-05 


volatile 
 0.0005987110104128758 


volatility 
 2.347886315344611e-05 


volcanic 
 0.0006808870314499372 


volcanicity 
 2.347886315344611e-05 


volcanism 
 1.1739431576723054e-05 


volcanist 
 1.1739431576723054e-05 


volcano 
 0.00022304919995773804 


volition 
 2.347886315344611e-05 


volley 
 1.1739431576723054e-05 


volleyball 
 1.1739431576723054e-05 


volt 
 8.217602103706138e-05 


voltage 
 0.00037566181045513774 


voltaic 
 2.347886315344611e-05 


voltmeter 
 1.1739431576723054e-05 


voluble 
 1.1739431576723054e-05 

zaire 
 1.1739431576723054e-05 


zamindar 
 1.1739431576723054e-05 


zander 
 1.1739431576723054e-05 


zeal 
 5.869715788361528e-05 


zealous 
 3.5218294730169165e-05 


zealously 
 3.5218294730169165e-05 


zebra 
 1.1739431576723054e-05 


zebu 
 1.1739431576723054e-05 


zedoary 
 1.1739431576723054e-05 


zenith 
 7.043658946033833e-05 


zeolite 
 8.217602103706138e-05 


zero 
 0.0003874012420318608 


zygoma 
 1.1739431576723054e-05 


zygomorphic 
 3.5218294730169165e-05 


zygospore 
 3.5218294730169165e-05 


zygote 
 0.00015261261049739973 


zygotene 
 1.1739431576723054e-05 


zigzag 
 0.0001291337473439536 


zinc 
 0.0003991406736085839 


zip 
 3.5218294730169165e-05 


zircon 
 3.5218294730169165e-05 


zirconium 
 8.217602103706138e-05 


zither 
 5.869715788361528e-05 


zloty 
 1.1739431576723054e-05 


zodiac 
 0.00017609147365084583 


zoetrope 
 1.1739431576723054e-05 


zoisite 
 1.1739431576723054e-05 


zonal 
 3.5218294730169165e-05 


zone 
 0.0003052252

In [85]:
for word in Core:
    Core_defining_power[word] = 0
    for value in dicty.values():
        if word in value:
            Core_defining_power[word] += 1
    Core_defining_power[word] /= len(Dictionary)    
    
find_contents(Core_defining_power)

a 
 0.5519293755796344 


class 
 0.004543160020191822 


conversation 
 0.0006221898735663219 


first 
 0.004813166946456453 


for 
 0.09690900766584883 


in 
 0.2646185271709144 


indicate 
 0.0014439500839369358 


membership 
 0.00017609147365084583 


of 
 0.5336276017515232 


or 
 0.5069790920723618 


people 
 0.01767958395454492 


per 
 0.001256119178709367 


person 
 0.07084746956552364 


someone 
 0.01979268163835507 


something 
 0.052944836411020976 


text 
 0.002582674946879072 


the 
 0.3846072573166007 


thing 
 0.015859972060152848 


time 
 0.010741579892701596 


to 
 0.26052146555063804 


with 
 0.12696195250225983 


aa 
 1.1739431576723054e-05 


pahoehoe 
 1.1739431576723054e-05 


abalone 
 2.347886315344611e-05 


ormer 
 1.1739431576723054e-05 


abandon 
 0.00016435204207412277 


desert 
 0.00037566181045513774 


abaxial 
 1.1739431576723054e-05 


adaxial 
 1.1739431576723054e-05 


abduct 
 4.695772630689222e-05 


adduct 
 2.347886315344611e-

 5.869715788361528e-05 


auricle 
 5.869715788361528e-05 


attach 
 0.00016435204207412277 


seize 
 0.0002465280631111842 


attached 
 0.0035687871993238087 


attempt 
 0.0006339293051430449 


effort 
 0.0011739431576723055 


attention 
 0.0020544005259265346 


care 
 0.0010448094103283519 


interesting 
 0.00045783783149219915 


notice 
 0.0006221898735663219 


attitude 
 0.0011269854313654133 


orientation 
 0.00032870408414824553 


attorney 
 5.869715788361528e-05 


barrister 
 0.00016435204207412277 


lawyer 
 9.391545261378444e-05 


solicitor 
 8.217602103706138e-05 


attract 
 0.0005869715788361528 


entice 
 7.043658946033833e-05 


attributive 
 0.0015730838312808895 


predicative 
 0.0005634927156827066 


aubergine 
 2.347886315344611e-05 


eggplant 
 1.1739431576723054e-05 


auction 
 0.00016435204207412277 


bid 
 0.0002934857894180764 


bidder 
 3.5218294730169165e-05 


aunt 
 3.5218294730169165e-05 


uncle 
 5.869715788361528e-05 


auriscope 
 1

 2.347886315344611e-05 


uniformitarianism 
 1.1739431576723054e-05 


category 
 0.0006926264630266602 


caterpillar 
 0.00027000692626463025 


larva 
 0.0006574081682964911 


catlinite 
 1.1739431576723054e-05 


pipestone 
 1.1739431576723054e-05 


catmint 
 2.347886315344611e-05 


catnip 
 1.1739431576723054e-05 


cattle 
 0.0011739431576723055 


ruminant 
 0.00022304919995773804 


phenomenon 
 0.0005282744209525375 


reason 
 0.00084523907352406 


cautionary 
 2.347886315344611e-05 


warning 
 0.0004695772630689222 


caviar 
 5.869715788361528e-05 


sturgeon 
 8.217602103706138e-05 


ceiba 
 1.1739431576723054e-05 


kapok 
 2.347886315344611e-05 


ceiling 
 0.00027000692626463025 


room 
 0.0024770200626885648 


cell 
 0.004437505136001315 


cytoplasm 
 0.0005517532841059836 


electrolyte 
 2.347886315344611e-05 


cellobiose 
 1.1739431576723054e-05 


cellose 
 1.1739431576723054e-05 


cellulifugal 
 2.347886315344611e-05 


cellulipetal 
 2.347886315344611

 0.0024887594942652877 


deduction 
 0.00014087317892067666 


induction 
 0.000211309768381015 


defamation 
 2.347886315344611e-05 


libel 
 7.043658946033833e-05 


defendant 
 0.0001291337473439536 


plaintiff 
 0.00014087317892067666 


defensible 
 1.1739431576723054e-05 


justifiable 
 8.217602103706138e-05 


deferral 
 3.5218294730169165e-05 


postponement 
 3.5218294730169165e-05 


definiendum 
 1.1739431576723054e-05 


definiens 
 1.1739431576723054e-05 


degree 
 0.003474871746710024 


temperature 
 0.0013265557681697052 


deictic 
 2.347886315344611e-05 


indexical 
 1.1739431576723054e-05 


deism 
 1.1739431576723054e-05 


theism 
 2.347886315344611e-05 


deity 
 0.0004930561262223683 


god 
 0.0017726541680851812 


goddess 
 0.0005165349893758144 


intentional 
 0.00011739431576723055 


deliberately 
 0.0006221898735663219 


intentionally 
 0.00016435204207412277 


delicate 
 0.0005165349893758144 


sensitive 
 0.0007161053261801063 


delirium 
 0.

freehold 
 0.00011739431576723055 


leasehold 
 1.1739431576723054e-05 


frequency 
 0.0007982813472171677 


frequently 
 0.0020309216627730883 


often 
 0.007161053261801064 


froghopper 
 1.1739431576723054e-05 


spittlebug 
 1.1739431576723054e-05 


journey 
 0.0006104504419895989 


material 
 0.009321108671918106 


prevention 
 0.00018783090522756887 


raw 
 0.0006104504419895989 


source 
 0.002195273704847211 


which 
 0.07972247983752627 


fructose 
 7.043658946033833e-05 


hexose 
 5.869715788361528e-05 


oil 
 0.0025357172205721797 


fugue 
 7.043658946033833e-05 


hysteria 
 4.695772630689222e-05 


plump 
 0.00022304919995773804 


fungus 
 0.001068288273481798 


yeast 
 0.00027000692626463025 


furcula 
 3.5218294730169165e-05 


wishbone 
 2.347886315344611e-05 


g 
 9.391545261378444e-05 


u 
 5.869715788361528e-05 


galactose 
 0.0001056548841905075 


lactose 
 3.5218294730169165e-05 


galea 
 2.347886315344611e-05 


helmet 
 0.000223049199957738

lumper 
 1.1739431576723054e-05 


splitter 
 1.1739431576723054e-05 


lumpfish 
 1.1739431576723054e-05 


lumpsucker 
 1.1739431576723054e-05 


sheen 
 9.391545261378444e-05 


machine 
 0.0028409424415669795 


mechanical 
 0.001056548841905075 


magazine 
 0.00042261953676203 


periodical 
 0.00011739431576723055 


readership 
 1.1739431576723054e-05 


magaziner 
 1.1739431576723054e-05 


magazinist 
 1.1739431576723054e-05 


magnitude 
 0.0002582674946879072 


maguey 
 3.5218294730169165e-05 


mescal 
 3.5218294730169165e-05 


may 
 0.004273153093927192 


possibility 
 0.0003991406736085839 


maintain 
 0.0004460983999154761 


preserve 
 0.00036392237887841473 


major 
 0.0015261261049739972 


minor 
 0.001291337473439536 


majuscule 
 1.1739431576723054e-05 


uncial 
 2.347886315344611e-05 


prepare 
 0.0003404435157249686 


malleable 
 0.0001056548841905075 


manage 
 0.0002817463578413533 


run 
 0.000868717936677506 


manganic 
 1.1739431576723054e-05 



restrike 
 1.1739431576723054e-05 


restriking 
 1.1739431576723054e-05 


rumen 
 5.869715788361528e-05 


vengeance 
 7.043658946033833e-05 


rheumatism 
 9.391545261378444e-05 


rheumatoid 
 3.5218294730169165e-05 


verse 
 0.001068288273481798 


rich 
 0.0012678586102860899 


wealthy 
 0.0003404435157249686 


ridge 
 0.0008334996419473369 


watershed 
 1.1739431576723054e-05 


whisky 
 0.00015261261049739973 


riel 
 1.1739431576723054e-05 


sen 
 4.695772630689222e-05 


ringgit 
 1.1739431576723054e-05 


ringworm 
 7.043658946033833e-05 


tinea 
 1.1739431576723054e-05 


river 
 0.0026413721047626876 


roll 
 0.0006691475998732142 


romantic 
 0.00042261953676203 


turbine 
 8.217602103706138e-05 


roture 
 2.347886315344611e-05 


roturier 
 1.1739431576723054e-05 


rousing 
 3.5218294730169165e-05 


stirring 
 0.00022304919995773804 


rum 
 0.00018783090522756887 


sheep 
 0.0012326403155559207 


rundale 
 1.1739431576723054e-05 


runrig 
 1.173943157672

In [86]:
def ShannonsCalculation(dictionary):
    information = 0
    for word, probability in dictionary.items():
        information += probability * math.log(probability, 2)
    return -(information)

print(ShannonsCalculation(Core_defining_power))
print(ShannonsCalculation(Kernel_defining_power))

31.83412502338212
66.46740151746502
